In [1]:
!pip install accelerate transformers==4.37.0 datasets

In [2]:
import torch
from datasets import load_dataset
from PIL import Image
from torchvision.io import ImageReadMode, read_image
from torchvision.transforms import CenterCrop, ConvertImageDtype, Normalize, Resize, RandomHorizontalFlip,RandomAffine,GaussianBlur,RandomCrop, RandomRotation, RandomResizedCrop, CenterCrop
from torchvision.transforms.functional import InterpolationMode
from transformers import (
    Trainer,
    TrainingArguments,
    VisionTextDualEncoderModel,
    VisionTextDualEncoderProcessor,
    AutoTokenizer,
    AutoImageProcessor
)

model = VisionTextDualEncoderModel.from_vision_text_pretrained(
    "google/vit-base-patch16-224-in21k", "princeton-nlp/sup-simcse-roberta-base"
)
#facebook/deit-base-distilled-patch16-384

tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-roberta-base")
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
processor = VisionTextDualEncoderProcessor(image_processor, tokenizer)
config = model.config

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The projection layer and logit scale weights `['visual_projection.weight', 'text_projection.weight', 'logit_scale']` are newly initialized. You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from datasets import load_dataset
file_path = "./vlm_caption.jsonl"
dataset = load_dataset('json', data_files=file_path)
dataset = dataset['train'].train_test_split(test_size=0.05, seed = 42)
dataset['valid'] = dataset['test'] 
dataset

DatasetDict({
    train: Dataset({
        features: ['image_path', 'image_id', 'original_id', 'caption'],
        num_rows: 26517
    })
    test: Dataset({
        features: ['image_path', 'image_id', 'original_id', 'caption'],
        num_rows: 1396
    })
    valid: Dataset({
        features: ['image_path', 'image_id', 'original_id', 'caption'],
        num_rows: 1396
    })
})

In [4]:
dataset['train'][0]

{'image_path': './cropped/40200.jpg',
 'image_id': 40200,
 'original_id': 2010,
 'caption': 'white and orange light aircraft'}

In [5]:
# We use torchvision for faster image pre-processing. The transforms are implemented as nn.Module,
# so we jit it to be faster.
class Transform(torch.nn.Module):
    def __init__(self, image_size, mean, std):
        super().__init__()
        self.transforms = torch.nn.Sequential(
            #Resize([image_size + 5], interpolation=InterpolationMode.BICUBIC),
            #RandomCrop(image_size),
            RandomResizedCrop(image_size, scale=(0.9,1.0),interpolation=InterpolationMode.BICUBIC),
            ConvertImageDtype(torch.float),
            RandomRotation(10),
            RandomHorizontalFlip(p=0.5),
            #GaussianBlur(kernel_size=3),
            Normalize(mean, std),
        )

    def forward(self, x) -> torch.Tensor:
        """`x` should be an instance of `PIL.Image.Image`"""
        with torch.no_grad():
            x = self.transforms(x)
        return x
    
    
class TestTransform(torch.nn.Module):
    def __init__(self, image_size, mean, std):
        super().__init__()
        self.transforms = torch.nn.Sequential(
            Resize([image_size], interpolation=InterpolationMode.BICUBIC),
            CenterCrop(image_size),
            #RandomResizedCrop(image_size, scale=(0.8,1.0),interpolation=InterpolationMode.BICUBIC),
            ConvertImageDtype(torch.float),
            #RandomRotation(90),
            #RandomHorizontalFlip(p=0.5),
            #GaussianBlur(kernel_size=3),
            Normalize(mean, std),
        )

    def forward(self, x) -> torch.Tensor:
        """`x` should be an instance of `PIL.Image.Image`"""
        with torch.no_grad():
            x = self.transforms(x)
        return x

# For preprocessing the datasets.
# Initialize torchvision transforms and jit it for faster processing.
transform = Transform(
    config.vision_config.image_size, image_processor.image_mean, image_processor.image_std
)
transform = torch.jit.script(transform)

test_transform = TestTransform(
    config.vision_config.image_size, image_processor.image_mean, image_processor.image_std
)
test_transform = torch.jit.script(test_transform)

In [6]:
def preprocess_dataset(dataset, split):
    # Preprocessing the datasets.
    data = dataset[split]
    # We need to tokenize inputs and targets.
    column_names = data.column_names

    # 6. Get the column names for input/target.
    image_column = "image_path"
    caption_column = "caption"
    dataset_columns = (image_column, caption_column)

    # Preprocessing the datasets.
    # We need to tokenize input captions and transform the images.
    def tokenize_captions(examples):
        captions = list(examples[caption_column])
        text_inputs = tokenizer(captions, padding="max_length", truncation=True)
        examples["input_ids"] = text_inputs.input_ids
        examples["attention_mask"] = text_inputs.attention_mask
        return examples

    def transform_images(examples):
        images = [read_image(image_file, mode=ImageReadMode.RGB) for image_file in examples[image_column]]
        if split == 'train':
            examples["pixel_values"] = [transform(image) for image in images]
        else:
            examples["pixel_values"] = [test_transform(image) for image in images]
        return examples

    data = data.map(
        function=tokenize_captions,
        batched=True,
        remove_columns=[col for col in column_names if col != image_column],
        desc=f"Running tokenizer on {split} dataset",
    )

    # Transform images on the fly as doing it on the whole dataset takes too much time.
    data.set_transform(transform_images)
    return data

In [7]:
train_dataset = preprocess_dataset(dataset, "train")
eval_dataset = preprocess_dataset(dataset, "valid")

Parameter 'transform'=<function preprocess_dataset.<locals>.transform_images at 0x7f13db69c670> of the transform datasets.arrow_dataset.Dataset.set_format couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [8]:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    input_ids = torch.tensor([example["input_ids"] for example in examples], dtype=torch.long)
    attention_mask = torch.tensor([example["attention_mask"] for example in examples], dtype=torch.long)
    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "return_loss": True,
    }

In [9]:
# initialize Trainer
training_args = TrainingArguments(
    learning_rate=1e-5,
    warmup_steps=0,
    weight_decay=0.02,
    evaluation_strategy = "epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    remove_unused_columns=False,
    output_dir="simcse-vit",
    save_strategy = "no",
    num_train_epochs=3,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn,
)
train_result = trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch,Training Loss,Validation Loss
1,0.094400,0.099741
2,0.055100,0.066080



KeyboardInterrupt



In [10]:
output_dir = 'caption_models/simcse-vit-v1'
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
image_processor.save_pretrained(output_dir)

['caption_models/simcse-vit-v1/preprocessor_config.json']

In [ ]:
metrics = trainer.evaluate()
print(metrics)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
idx = 0
image = [Image.open(dataset['train'][idx]['image_path'])]
texts = [dataset['train'][idx]['caption'], "black helicopter", "black fighter plane", "white missile"]
print(texts)
inputs = processor(text=texts, images=image, return_tensors="pt", padding=True).to(device)
model = model.to(device)
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image.sigmoid() 
print(logits_per_image)
image[0]